# Docs

Generate documentation for the project. Extract MD cells from all the notebooks
and turn them into a doc site powered by [Vitepress](https://vitepress.dev/)
hosted on Github Pages.

In [2]:
//| export

// import { getConfig, getTestConfig } from "jurassic/config.ts";